# Import libraries

In [4]:

import requests
import pyodbc
import random
from dotenv import load_dotenv
import os

# Global Variables 

In [ ]:
load_dotenv()
DRIVER= os.getenv("DRIVER")
SERVER=os.getenv("SERVER")
DATABASE=os.getenv('DATABASE')
Trusted_Connection=os.getenv('Trusted_Connection')
Base_url=os.getenv('Base_url')
key=os.getenv('key')


7e956bd54f1c415a91251763f6447426


# Conecting SQL SERVER

In [13]:

# Connect to SQL Server (Global variables)
conn = pyodbc.connect(f'DRIVER={DRIVER};'
                      f'SERVER={SERVER}'
                      f'DATABASE={DATABASE};'
                     f'Trusted_Connection={Trusted_Connection};'
                                              )

cursor = conn.cursor()

# function:product_exists

In [55]:
def product_exists(product_id,Table):
    query = f"SELECT TOP 1 1 FROM {Table} WHERE id = ?"
    cursor.execute(query, (product_id,))
    return cursor.fetchone() is not None

# function: Inserting products data Api into Sql Products table:

In [56]:
#---- i didnt finish
def insert_products(Products_info,Table):
    new_products=[]
    for product in Products_info:
        if not product_exists(product['id'],Table):
            new_products.append((
                product['id'],
                product['title'],
                product['category'],
                product['description'],
                float(product['price'])
            ))
    

    insert_query = """
    INSERT INTO SILVER.Store_Products (id, title, category, description,price)
    VALUES (?, ?, ?, ?,?)
    """
    cursor.executemany(insert_query, new_products)
    conn.commit()

# extraction of Users,Carts, Products from Api

In [57]:
def Pulling_Store_Api_Data():
    # Define the API endpoint
    Base_url = "https://fakestoreapi.com/"
    entites_list=['products','carts','users']
    # Pulling  all the relevent information of the entity
    All_data={}

    # Send the GET request
    for entity in entites_list:
        response = requests.get(Base_url+entity)
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            All_data[entity]=data
        # Print the JSON data
    return All_data


# Extracation of geographic information

In [15]:
# Define the API endpoint(Q=latitude,longitude)
def Pulling_Geographic_Api_Data(latitude,longitude):
    url = f"https://api.opencagedata.com/geocode/v1/json?q={latitude}%2C{longitude}&key={key}"
    response_product = requests.get(url) 
    # Check if the request was successful (status code 200)
    if response_product.status_code == 200:
        # Parse the JSON response
        data_ = response_product.json()
        # Print the JSON data
        print(data_)
        # extract relevent columns
        city=data_['results'][0]['components']['_normalized_city']
        continent=data_['results'][0]['components']['continent']
        country=data_['results'][0]['components']['country']

        return city,continent,country

# function: Inserting Users data Api into Sql Products table:

In [59]:
def User_exists(User_id,Table):
    query = f"SELECT TOP 1 1 FROM {Table} WHERE userid = ?"
    cursor.execute(query, (User_id,))
    return cursor.fetchone() is not None

In [60]:
def insert_Users(Users_Info,Table):
    new_users=[]
    Coordinates=[(31.0461,34.8516),(52.3555,1.1743),(36.2048,138.2529),(51.1657,10.4515)]
    for User in Users_Info:
        if not User_exists(User['id'],Table):
            loacation= Coordinates[random.randint(0,3)]

            city,continent,country=Pulling_Geographic_Api_Data(loacation[0],loacation[1])
            new_users.append((
                User['id'],
                User['name']['firstname'],
                User['name']['lastname'],
                User['email'],
                city,
                continent,
                country

            )
            )
    

    insert_query = """
    INSERT INTO  SILVER.STORE_Users (userid, firstname, lastname, email,city,continent,country)
    VALUES (?, ?, ?, ?,?,?,?)
    """
    cursor.executemany(insert_query, new_users)
    conn.commit()

In [61]:
def Cart_exists(User_id,Table):
    query = f"SELECT TOP 1 1 FROM {Table} WHERE cart_id = ?"
    cursor.execute(query, (User_id,))
    return cursor.fetchone() is not None

In [62]:
def insert_Carts(Carts_Info,Table):
    new_Carts=[]
    for Cart in Carts_Info:
        if not Cart_exists(Cart['id'],Table):
            Cart_id=Cart['id']
            userId=Cart['userId']
            Cart_date= Cart['date']
            for product in Cart['products']:
                new_Carts.append((Cart_id,userId,Cart_date,product['productId'],product['quantity']))



                    

    insert_query = """
    INSERT INTO  SILVER.Store_Cart_Sales (cart_id, userid, purchase_date, item,quantity)
    VALUES (?, ?, ?, ?,?)
    """
    cursor.executemany(insert_query, new_Carts)
    conn.commit()

# Main :Inserting data into the  database

In [63]:
# pulling data from Store Api
All_data=Pulling_Store_Api_Data()
# inserting product data to database
insert_products(All_data['products'],'SILVER.Store_Products')
# inserting Users data to database
insert_Users(All_data['users'],'SILVER.STORE_Users')
# inserting carts to database
insert_Carts(All_data['carts'],'SILVER.Store_Cart_Sales')


{'documentation': 'https://opencagedata.com/api', 'licenses': [{'name': 'see attribution guide', 'url': 'https://opencagedata.com/credits'}], 'rate': {'limit': 2500, 'remaining': 2462, 'reset': 1740009600}, 'results': [{'annotations': {'DMS': {'lat': "52° 21' 25.05096'' N", 'lng': "1° 10' 27.22584'' E"}, 'MGRS': '31UCU7566402310', 'Maidenhead': 'JO02oi05vq', 'Mercator': {'x': 130714.62, 'y': 6831070.512}, 'OSM': {'edit_url': 'https://www.openstreetmap.org/edit?way=61690343#map=17/52.35696/1.17423', 'note_url': 'https://www.openstreetmap.org/note/new#map=17/52.35696/1.17423&layers=N', 'url': 'https://www.openstreetmap.org/?mlat=52.35696&mlon=1.17423#map=17/52.35696/1.17423'}, 'UN_M49': {'regions': {'EUROPE': '150', 'GB': '826', 'NORTHERN_EUROPE': '154', 'WORLD': '001'}, 'statistical_groupings': ['MEDC']}, 'callingcode': 44, 'currency': {'alternate_symbols': [], 'decimal_mark': '.', 'html_entity': '&#x00A3;', 'iso_code': 'GBP', 'iso_numeric': '826', 'name': 'British Pound', 'smallest_den